       Modelo de aprendizaje automático:

Modelo de aprendizaje automático: Para este modelo el input será un juego y el output una lista de 5 juegos recomendados, 

para ello usaremos la similitud del coseno. 

Vamos a tomar en cuenta el título, el juego y el precio del mismo para realizar este modelo, ya que ayudan a generar recomendaciones relevantes 

y personalizadas para los  usuarios abordando aspectos clave de las preferencias del usuario.

In [11]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fastapi import FastAPI

In [12]:
df_games = pd.read_csv('data/df_games_clean.csv', encoding='utf-8')
df_games.shape

(27617, 6)

In [13]:
# Tomar los primeros 1000 datos de steam_games
muestra = df_games.head(1000)

# Preprocesamiento de datos
steam = muestra.dropna(subset=['title', 'genres'])
steam['price'] = steam['price'].fillna(0)
steam['genres'] = steam['genres'].fillna('[]').astype(str)

# Combinamos las columnas 'genres' y 'title' en una sola columna 'features'
steam['features'] = steam['genres'] + ' ' + steam['title']

# Inicializamos el vectorizador de texto
vectorizer = CountVectorizer()

# Creamos una matriz de términos-documentos
X = vectorizer.fit_transform(steam['features'])

# Calculamos la similitud del coseno entre los juegos
cosine_sim = cosine_similarity(X, X)

def recomendacion_juego(game_id):
    try:
        # Obtener el título del juego a partir del ID
        game_title = steam.loc[steam['id'] == game_id, 'title'].values[0]

        # Obtener el índice del juego en el DataFrame
        game_index = steam.index[steam['title'] == game_title].tolist()[0]

        # Obtener las puntuaciones de similitud del juego en cuestión
        sim_scores = list(enumerate(cosine_sim[game_index]))

        # Ordenar los juegos por similitud en orden descendente
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtener los 5 juegos más similares (excluyendo el juego de entrada)
        top_n = 5
        recommended_games = []
        for i, sim in sim_scores[1:top_n+1]:
            recommended_games.append(steam['title'].iloc[i])

        return {"Juego": game_title, "juegos_recomendados": recommended_games}
    except Exception as e:
        return {"error": str(e)}

Resultados:

In [14]:
game_id = 670290	 # Pongo el ID del juego que deseo buscar recomendaciones
recomendaciones = recomendacion_juego(game_id)
print(recomendaciones)

{'Juego': 'Real Pool 3D - Poolians', 'juegos_recomendados': ['SNOW - Starter Pack', 'SNOW - Lifetime Pack', 'SNOW - All Access Basic Pass', 'SNOW - All Access Pro Pass', 'SNOW - All Access Legend Pass']}
